# Locking of Timoschenko beam

In [1]:
from ngsolve import *
from ngsolve.meshes import Make1DMesh
from ngsolve.webgui import Draw

importing NGSolve-6.2.2204


In [2]:
# use material parameters such that equations simplify
mu, lam = 0.5, 0  # Lame parameter
nu = lam/(2*(lam+mu)) # Possion ratio (=0)
E = mu*(3*lam+2*mu)/(lam+mu) # Young's modulus (=1)
k = 5/6           # shear correction factor
G = E/(2*(1+nu))  # shearing modulus
alpha= 1/(12*k*G)

In [3]:
from sympy import symbols, simplify, solve, diff, lambdify

def ComputeExactSolution(t=0.01, q=-0.1, shearforce=True):
    X, C1, C2, C3, C4 = symbols("x C1 C2 C3 C4")
    if shearforce:
        w_ex = -C1/6*X**3-C2/2*X**2 + C3*X+C4
        w_diff_ex = -C1/2*X**2-C2*X + C3
        beta_ex = -C1/2*X**2-C2*X-alpha*t**2*C1+C3
        beta_diff_ex = -C1*X-C2

        solve_bc = solve((w_ex.subs(X,0), (k*G/t**2*(w_diff_ex-beta_ex)-q).subs(X,1), beta_ex.subs(X,0), beta_diff_ex.subs(X,1)), C1, C2, C3, C4)
    else: # uniform volume force
        w_ex = q/2*X**4-C1/6*X**3-0.5*(C2+alpha*t**2*q*12)*X**2+C3*X+C4
        w_diff_ex = 2*q*X**3-C1/2*X**2-(C2+alpha*t**2*q*12)*X+C3
        beta_ex = 2*q*X**3-C1/2*X**2-C2*X-alpha*t**2*C1+C3
        beta_diff_ex = q*12/2*X**2-C1*X-C2
        solve_bc = solve((w_ex.subs(X,0), (w_diff_ex-beta_ex).subs(X,1), beta_ex.subs(X,0), beta_diff_ex.subs(X,1)), C1, C2, C3, C4)
    c1,c2,c3,c4 = solve_bc[C1], solve_bc[C2], solve_bc[C3], solve_bc[C4]

    # generate CoefficientFunctions
    w_ex_func = lambdify((X), w_ex.subs([(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
    cf_w_ex = w_ex_func(x)
    beta_ex_func = lambdify((X), beta_ex.subs([(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
    cf_beta_ex = beta_ex_func(x)
    
    return (cf_w_ex, cf_beta_ex)

In [4]:
cf_w_ex, cf_beta_ex = ComputeExactSolution(t=0.01,q=-0.1)
mesh = Make1DMesh(10)
Draw(cf_w_ex, mesh, deformation=CF( (0,cf_w_ex,0) ))

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

BaseWebGuiScene

In [5]:
def SolveTB(order, mesh, t, q, shearforce=True, w_ex=None, beta_ex=None,\
            reduced_integration=False, increase_Wspace=False, draw=False):
    fes = H1(mesh, order=order+increase_Wspace, dirichlet="left")*H1(mesh, order=order, dirichlet="left") 
    (w,beta),(dw,dbeta) = fes.TnT()
    
    a = BilinearForm(fes, symmetric=True)
    a += 1/12*grad(beta)*grad(dbeta)*dx \
        + k*G/t**2*(grad(w)-beta)*(grad(dw)-dbeta)*dx(bonus_intorder=-reduced_integration)
    
    f = LinearForm(fes)
    if shearforce:
        f += q*dw*ds("right")
    else:
        f += q*dw*dx
        
    gfsol = GridFunction(fes)
    
    a.Assemble()
    f.Assemble()
    gfsol.vec.data = a.mat.Inverse(fes.FreeDofs(),inverse="sparsecholesky")*f.vec
    gfw, gfbeta = gfsol.components
    
    if draw:
        Draw(gfw,mesh, deformation=CF((0,gfw,0)))
        Draw(gfbeta,mesh, deformation=CF((0,gfbeta,0)))
    
    errW = 1
    errbeta = 1
    if w_ex:
        errW = sqrt(Integrate((gfw-cf_w_ex)**2+(Grad(gfw)-cf_w_ex.Diff(x))**2,mesh))/sqrt(Integrate(cf_w_ex**2+cf_w_ex.Diff(x)**2,mesh))
    if beta_ex:
        errbeta = sqrt(Integrate((gfbeta-cf_beta_ex)**2+(Grad(gfbeta)-cf_beta_ex.Diff(x))**2,mesh))/sqrt(Integrate(cf_beta_ex**2+cf_beta_ex.Diff(x)**2,mesh))
    return (errW, errbeta)

In [6]:
shearforce=False

ts = [1,1e-1,1e-2,1e-3]
hs = [0.5**i for i in range(1,8)]

errW = []
errbeta = []
with TaskManager():
    for t in ts:
        cf_w_ex, cf_beta_ex =ComputeExactSolution(t=t, q=-0.1, shearforce=shearforce)
        #Draw(cf_w_ex, mesh, deformation=CF( (0,cf_w_ex,0) ))
        errW.append([])
        errbeta.append([])
        for h in hs:
            mesh=Make1DMesh(int(1/h))
            errw,errb = SolveTB(order=1, mesh=mesh, t=t, q=-0.1, shearforce=shearforce, \
                                w_ex=cf_w_ex, beta_ex=cf_beta_ex, reduced_integration=False,\
                                increase_Wspace=False)
            errW[-1].append(errw)
            errbeta[-1].append(errb)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

...

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

In [ ]:
import matplotlib.pyplot as plt
plt.yscale('log')
plt.xscale('log')
plt.xlabel("t")
plt.ylabel("relative err")
for i in range(len(ts)):
    plt.plot(hs, errW[i], "*-", label="$\|w-w_{\mathrm{ex}}\|$ $t="+str(ts[i])+"$")
    plt.plot(hs, errbeta[i], "*-", label="$\|b-b_{\mathrm{ex}}\|$ $t="+str(ts[i])+"$")
    
plt.plot(hs, hs, '-' , color="k", label="$\mathcal{O}(h)$")
plt.plot(hs, [h**2 for h in hs], '-' , color="k", label="$\mathcal{O}(h^2)$")
plt.legend()
plt.show()